# EarthNet2021 Model Intercomparson Suite Cookbook

Run.py is the main entry point. You should be able to launch any task from it! But first let's...

## Download EarthNet2021 Dataset

In [ ]:
import earthnet as en

en.Downloader.get('data/datasets/release', 'all') #["train","iid","ood","extreme","seasonal"]

## Test Baseline

In [ ]:
!python run.py --model_name Baseline --split_name iid_test_split/context --submodel mean

## Train
The EN Intercomparison suite comes with a Tensorflow baseline based on SAVP that we call Arcon-STF (Autorgressive Conditional Spatio-Temporal Forecaster).
You can train and test any model using run.py. Checkout 'configs/tf_template/' to learn how the standalone model is passed arguments from run.py.

In [ ]:
# Firt we generate tfrecords samples for the train set using the included util
!source activate ENtf115py36
!python utils/npz_to_tfecords.py --inpath data/datasets/release/train/ --outpath data/tf_dataset/train

In [ ]:
# Running train Arcon trough run.py

!python run.py --model_name tf_template --dataroot data/tf_dataset/ --split_name train --experiment_name 02_Arcon_earthnet --mode train --submodel arcon_earthnet --dataset earthnet --gpu_ids 3 --experiment_settings 02-testing-earthnet

In [ ]:
#To continue training a paused train run

!python run.py --model_name Arcon --dataroot data/tf_dataset/ --split_name train --experiment_name 02_Arcon_earthnet --mode train --submodel arcon_earthnet --dataset earthnet --gpu_ids 3 --resume

## Test

In [ ]:
#Predicting Arcon 

!python run.py --model_name Arcon --mode test --gpu_ids 3 --experiment_name 02_Arcon_earthnet --submodel arcon_earthnet --dataset earthnet --split_name iid_test --dataroot data/tf_dataset/ --num_stochastic_samples 1 --batch_size 1

In [ ]:
#Predicting Baseline model linear regression

!python run.py --model_name Baseline --split_name iid_test --submodel linear

## Evaluate

In [ ]:
#Evaluates the output of the model tested
!python run.py --mode evaluate --model_name Baseline --experiment_name mean

#Many model / experiments
!python run.py --mode evaluate --model_name Baseline Baseline Arcon Arcon --experiment_name mean linear 02_Arcon_earthnet 02_Arcon_earthnet_210000

In [ ]:
#plotting the figures to display evaluation results
!python run.py --mode plot

## Plot
utils/plot_sample.py

In [ ]:
# Plotting a single frame from NPZ with predictions

!python utils/plot_sample.py --pred --frame 150 --image

In [ ]:
# Plotting a .mp4 video

!python utils/plot_sample.py --pred --frame 150 --video --predpath data/outputs/Arcon/02_Arcon_earthnet/iid_test/ 

#Plotting predictions whose groundtruth is split into context and target files, doesn't work yet.
!python utils/plot_sample.py --pred --frame 150 --video --predpath data/outputs/Baseline/mean/ood_test_split/context/ --tilename 29TNE --filename context_29TNE_2017-06-20_2017-11-16_3129_3257_5049_5177_24_104_39_119.npz --datapath data/datasets/ood_test_split/context/